In [1]:
import gradio as gr
import faiss
import pandas as pd
import numpy as np
from openai import AzureOpenAI

# 1. Azure OpenAI 클라이언트 설정
client = AzureOpenAI(
    api_key="YOUR_AZURE_OPENAI_KEY",  # 실제 키로 교체
    api_version="2024-05-01-preview",
    azure_endpoint="https://team05-openai.openai.azure.com"
)

# 2. FAISS 인덱스 및 메타데이터 로드
df_meta = pd.read_csv("FAISS/faiss_metadata.csv", encoding="utf-8-sig")
index = faiss.read_index("FAISS/faiss_index.idx")

# 3. 질문 처리 함수
def answer_question(query):
    try:
        # 질문 임베딩
        resp = client.embeddings.create(
            input=[query],
            model="text-embedding-3-large"
        )
        query_vec = np.array(resp.data[0].embedding, dtype='float32').reshape(1, -1)

        # FAISS로 유사 chunk 검색
        D, I = index.search(query_vec, k=3)
        context = "\n\n".join(df_meta.loc[i, "chunk"] for i in I[0])

        # GPT로 응답 생성
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role": "system",
                    "content": """다음은 보험 약관 전문 AI입니다. 아래 약관을 바탕으로 질문에 다음의 **고정된 형식**으로 답변하세요:

[보장 여부]   
[필요 서류]  
[적용 조건]  

항목 이름과 이모지는 반드시 그대로 유지하세요.  
다른 양식(번호 매기기, 자유서술 등)은 절대 사용하지 마세요.  
항목이 없는 경우 '약관에 명시되지 않음' 또는 '해당 없음'으로 반드시 채워주세요.  
답변 마지막에 반드시 참고한 약관 내용을 "📎 약관 근거:"로 요약해서 보여주세요."""
                },
                {
                    "role": "user",
                    "content": f"문서:\n{context}\n\n질문: {query}"
                }
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        return f"❌ 오류 발생: {str(e)}"

# 4. Gradio UI 구성
with gr.Blocks() as demo:
    gr.Markdown("## 🛡️ 보험 약관 상담 챗봇")
    with gr.Row():
        query_input = gr.Textbox(label="💬 질문을 입력하세요", placeholder="예: 아이가 친구와 싸워서 다치면 보험 적용되나요?")
    with gr.Row():
        output = gr.Textbox(label="📄 AI 응답", lines=10)
    submit_btn = gr.Button("질문하기")
    submit_btn.click(fn=answer_question, inputs=query_input, outputs=output)

# 5. 실행
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://d8781f68d90eb2d285.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
